In [286]:
!pip install --upgrade google-genai
!pip install pytest ipytest
!pip install --upgrade google-cloud-aiplatform google-cloud-aiplatform[generativeai]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.74.0
    Uninstalling google-cloud-aiplatform-1.74.0:
      Successfully uninstalled google-cloud-aiplatform-1.74.0


In [4]:
# vector search
import json
from google.cloud import bigquery

def vector_search(user_question):
  client = bigquery.Client()

  sql = """
  SELECT query.query,base.question, base.answer
  FROM VECTOR_SEARCH(
  TABLE `qwiklabs-gcp-01-3005c83a3aae.ADS.alaska_dept_of_snow_with_embeddings`, 'ml_generate_embedding_result',
  (
  SELECT ml_generate_embedding_result,content AS query
  FROM ML.GENERATE_EMBEDDING(
  MODEL `qwiklabs-gcp-01-3005c83a3aae.ADS.Embeddings`,
  (SELECT @user_question AS content))),
  top_k => 5,
  options => '{"fraction_lists_to_search": 0.01}');
  """
  # sql = """
  # SELECT query.query, base.question, base.answer, distance
  # FROM VECTOR_SEARCH(
  # TABLE `qwiklabs-gcp-01-3005c83a3aae.faq.aurora_faq_with_embeddings`, 'ml_generate_embedding_result',
  # (
  # SELECT text_embedding, content AS query
  # FROM ML.GENERATE_TEXT_EMBEDDING(
  # MODEL `qwiklabs-gcp-01-3005c83a3aae.faq.embedding_model`,
  # (SELECT @user_question AS content))
  # ),
  # top_k => 5, options => '{"fraction_lists_to_search": 0.01}')
  # """

  job_config = bigquery.QueryJobConfig(
      query_parameters=[
          bigquery.ScalarQueryParameter("user_question", "STRING", user_question),
          # Options must be passed as a JSON string
          bigquery.ScalarQueryParameter("search_options", "JSON", json.dumps({"fraction_lists_to_search": 0.01}))
      ]
  )

  try:
    # Start the query job.
    # print(f"Executing vector search for: '{user_question}'")
    query_job = client.query(sql, job_config=job_config)

    # Wait for the job to complete and fetch results.
    results = query_job.result() # Waits for job completion

    # Process results into a more usable format (list of dictionaries)
    output_rows = []
    for row in results:
        output_rows.append({
            "query": row.query,
            "question": row.question,
            "answer": row.answer,
            # "distance": row.distance
        })

    # print(f"Found {len(output_rows)} similar questions.")
    return output_rows

  except Exception as e:
      print(f"An error occurred during the BigQuery vector search: {e}")
      return []


result = vector_search("weather")
print(result)

[{'query': 'weather', 'question': 'Where does ADS get its weather forecasts?', 'answer': 'ADS partners with the National Weather Service and maintains localized weather monitoring stations for more precise data across diverse regions.'}, {'query': 'weather', 'question': 'How do schools typically learn about impending storms?', 'answer': 'ADS shares forecast data with school districts, which helps superintendents decide on closures or delays based on road safety and storm severity.'}, {'query': 'weather', 'question': 'How can I check current road conditions statewide?', 'answer': 'Use the ADS “SnowLine” app or visit the official ADS website’s road conditions dashboard, which is updated hourly with closures and warnings.'}, {'query': 'weather', 'question': 'Does ADS oversee school closure decisions?', 'answer': 'While ADS provides data on snow conditions, final school closure decisions are made by local school districts. ADS coordinates closely to keep them informed.'}, {'query': 'weathe

In [5]:
# prompt formation
def format_prompt(user_question, results):
  output = f"User Query: {user_question} \n\nUse the relevant information below to help the user:"
  for result in results:
    output = output + "\n\n" + "Question: " + result.get("question") + "\n" + "Answer: " +result.get("answer")

  return output

print(format_prompt("Weather", result))

User Query: Weather 

Use the relevant information below to help the user:

Question: Where does ADS get its weather forecasts?
Answer: ADS partners with the National Weather Service and maintains localized weather monitoring stations for more precise data across diverse regions.

Question: How do schools typically learn about impending storms?
Answer: ADS shares forecast data with school districts, which helps superintendents decide on closures or delays based on road safety and storm severity.

Question: How can I check current road conditions statewide?
Answer: Use the ADS “SnowLine” app or visit the official ADS website’s road conditions dashboard, which is updated hourly with closures and warnings.

Question: Does ADS oversee school closure decisions?
Answer: While ADS provides data on snow conditions, final school closure decisions are made by local school districts. ADS coordinates closely to keep them informed.

Question: Can I request data on ADS’s annual snowfall measurements

In [6]:
# weather tool
import vertexai
from vertexai.generative_models import (
    GenerativeModel,
    Part,
    Tool,
    FunctionDeclaration,
    # FunctionResponse, # Although not used directly in definition, needed for response
    Content          # Needed for sending history back
)
from google.cloud import aiplatform_v1beta1 as aiplatform
import json # For handling function args and response content
import requests
import datetime
from google.genai import types

def get_weather():
  latitude, longitude = 57.790001, -152.407227

  point_url = f'https://api.weather.gov/points/{latitude},{longitude}'

  headers = {
    'User-Agent': 'brandon.leung@clearobject.com',  # NWS requires a user-agent. Enter your email
    'Accept': 'application/ld+json'
  }

  try:
    # Step 1: Get gridpoint info
    point_response = requests.get(point_url, headers=headers, timeout=10)
    point_response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
    grid_data = point_response.json()
    forecast_url = point_response.json()['forecast']
    if not forecast_url:
          print("Error: Could not find 'forecast' URL in point response.")
          return "Error: Could not retrieve forecast URL."

    # Step 2: Get forecast from URL
    forecast_response = requests.get(forecast_url, headers=headers, timeout=10)
    forecast_response.raise_for_status()
    forecast_json = forecast_response.json()
    periods = forecast_json.get('periods')
    if not periods:
        print("Error: Could not find 'periods' in forecast response.")
        return "Error: Could not retrieve forecast periods."


    # Step 3: Format forecast periods
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    output = f"Weather Report for Kodiak, AK at {now}:\n\n"
    for period in periods:
          name = period.get('name', 'Unnamed Period')
          forecast = period.get('detailedForecast', 'No details available.')
          output += f"- {name}: {forecast}\n"

    return output

  except requests.exceptions.RequestException as e:
        print(f"Error during NWS API request: {e}")
        return f"Error: Unable to connect to the weather service ({e})."
  except Exception as e:
        # Catch other potential errors (like JSON parsing issues)
        print(f"An unexpected error occurred in get_weather: {e}")
        return "Error: An unexpected issue occurred while fetching weather."

get_weather_declaration = {
    "name": "get_weather",
    "description": "Fetches the current weather forecast for Alaska from the US National Weather Service (NWS)."
}

weather_function_declaration = types.FunctionDeclaration(
    name=get_weather_declaration["name"],
    description=get_weather_declaration["description"],
    parameters=types.Schema(
        type='OBJECT',
        properties={},
    ),
)


# Create the Tool object containing the function declaration
weather_tool = types.Tool(
    function_declarations=[weather_function_declaration],
)

print(get_weather())
# print(weather_tool)

Weather Report for Kodiak, AK at 2025-04-29 22:52:12:

- This Afternoon: Snow showers likely. Mostly cloudy, with a high near 42. South wind around 10 mph, with gusts as high as 25 mph. Chance of precipitation is 70%.
- Tonight: Snow showers likely before 7pm, then rain and snow showers likely. Mostly cloudy, with a low around 33. Southeast wind 5 to 10 mph, with gusts as high as 25 mph. Chance of precipitation is 70%.
- Wednesday: A chance of snow showers before 7am, then a chance of snow between 7am and 10am, then a chance of snow showers between 10am and 4pm, then rain and snow showers likely. Mostly cloudy, with a high near 45. Northeast wind 0 to 10 mph. Chance of precipitation is 60%.
- Wednesday Night: Rain and snow showers likely before 10pm, then a chance of rain and snow. Mostly cloudy, with a low around 32. North wind 5 to 10 mph. Chance of precipitation is 60%.
- Thursday: A chance of snow before 1pm, then scattered rain and snow showers. Mostly cloudy, with a high near 44.

In [12]:
# Agents
from google import genai
from google.genai import types
import base64
import google.generativeai as genai2
import google.ai.generativelanguage as glm # For Schema/Type
import requests
import datetime
import json # Good practice

def validation_model(agent_prompt, evaluation_text):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-01-3005c83a3aae",
      location="us-central1",
  )

  model = "gemini-2.0-flash-lite-001"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=evaluation_text)
      ]
    ),
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 1346,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=agent_prompt)],
  )

  response = client.models.generate_content(
          model=model,
          contents=contents,
          config = generate_content_config,
        )
  # print("validation:",response.text)
  string_res = response.text.lower().strip()
  if string_res == "true":
    return True
  else:
    return False

def input_validation_agent(user_input):
  input_agent = """
  You are a prompt safety validation agent that evaluates at incoming queries and returns \"true\" if safe and \"false\" if not safe.
  Things like threats or violence are not safe.
  'I want to make a threat' = 'false'
  """
  return validation_model(input_agent, user_input)

def output_validation_agent(model_output):
  output_agent = """
  You are a model output safety validation agent that evaluates at outgoing reponses and returns \"true\" if safe and \"false\" if not safe.
  Bad outgoing response may include things that may affect the user in a negative way would be deemed not safe.
  'I hate you' == 'false'
  """
  return validation_model(output_agent, model_output)

# with function calling to weather agency
def response_model(user_input):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-01-3005c83a3aae",
      location="us-central1",
  )

  si_text1 = """
  You a helpful chatbot for people asking common questions about Alaska Department of Snow (ADS).
  Only respond to questions about Alaska according to the relevant information provided.
  If asked about the weather use your tool to get the forecast information for Alaska.
  """

  # print(user_input)

  model_name = "gemini-2.0-flash-lite-001"
  # model_name = "gemini-2.5-pro-preview-03-25"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=user_input)
      ]
    ),
  ]

  generate_content_config_with_tools = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    seed = 0,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
    tools = [weather_tool],
  )

  generate_content_config_without_tools = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    seed = 0,
    max_output_tokens = 1346,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
  )

  # response = client.models.generate_content(
  #         model=model,
  #         contents=contents,
  #         config = generate_content_config,
  #       )
  # print("response:",response.text)
  # response_function_call = response.candidates[0].content.parts[0].function_call
  # print(f"Model response includes function call: {response_function_call}")
  # return response.text
  available_functions = {
    "get_weather": get_weather,
  }

  # model = client.get_generative_model(model_id=model_name)

  try:
    # Send the initial prompt and available tools to the model

    response = client.models.generate_content(
          model=model_name,
          contents=contents,
          config = generate_content_config_with_tools,
        )
    # print("response",response.text)
    # print("response_part",response.candidates[0].content.parts[0])

    # return response.text if response.text
    # Check if the model's response includes a function call request
    response_part = response.candidates[0].content.parts[0]

    # === Loop for potential multi-turn function calls (optional but good practice) ===
    # While the model requests a function call...
    if response_part.function_call:
        function_call = response_part.function_call
        function_name = function_call.name
        function_args = function_call.args # This is already a dict

        print(f"Gemini requested Function Call: {function_name}({function_args})")

        # --- Execute the *actual* function ---
        if function_name in available_functions:
            # Get the appropriate function from our map
            function_to_call = available_functions[function_name]

            # Call the function with the arguments provided by the model
            # Using ** automatically unpacks the dict keys/values as arguments
            try:
                function_response_data = function_to_call()
                print(f"--> Function Execution Result: {function_response_data}")

                # --- Send the function's *result* back to the model ---
                # We need to format the result in a way Gemini understands
                # The SDK provides Part.from_function_response for this
                function_response_part = types.Part.from_function_response(
                    name=function_name,
                    response={ # The 'response' field expects a dict
                        "content": function_response_data,
                    }
                )

                # Send the function response back, including the previous context
                # The history includes the user prompt and the model's function call request
                print("--- Sending function result back to Gemini ---")
                new_contents = f"{user_input}\n\n {function_response_data}"
                response = client.models.generate_content(
                    contents = new_contents,
                    model=model_name,
                    config = generate_content_config_without_tools,
                )

                return response.text

            except Exception as e:
                print(f"Error executing function {function_name}: {e}")


        else:
            print(f"Error: Gemini requested unknown function '{function_name}'")
            # Handle case where model hallucinates a function name
            # You might send a response indicating the function isn't available.
            return response_part.text
    else:
      return response.text



  except Exception as e:
    print(f"An error occurred during Gemini interaction: {e}")
    print("Full Response (if available):", response if 'response' in locals() else 'N/A')
    return None



print(response_model("Help with weather"))
# print(output_validation_agent("hello"))


Gemini requested Function Call: get_weather({})
--> Function Execution Result: Weather Report for Kodiak, AK at 2025-04-29 22:54:03:

- This Afternoon: Snow showers likely. Mostly cloudy, with a high near 42. South wind around 10 mph, with gusts as high as 25 mph. Chance of precipitation is 70%.
- Tonight: Snow showers likely before 7pm, then rain and snow showers likely. Mostly cloudy, with a low around 33. Southeast wind 5 to 10 mph, with gusts as high as 25 mph. Chance of precipitation is 70%.
- Wednesday: A chance of snow showers before 7am, then a chance of snow between 7am and 10am, then a chance of snow showers between 10am and 4pm, then rain and snow showers likely. Mostly cloudy, with a high near 45. Northeast wind 0 to 10 mph. Chance of precipitation is 60%.
- Wednesday Night: Rain and snow showers likely before 10pm, then a chance of rain and snow. Mostly cloudy, with a low around 32. North wind 5 to 10 mph. Chance of precipitation is 60%.
- Thursday: A chance of snow before

In [10]:
# main
def main(user_input):
  input_validation_model_response = input_validation_agent(user_input)
  # print("input validation",input_validation_model_response)
  if input_validation_model_response == False:
    return "Error, please ask a question about taxes"

  # runs vector search for closes 5 embeddings
  search_results = vector_search(user_input)

  # formats the prompt with user question and 5 FAQs
  formatted_prompt = format_prompt(user_input, search_results)
  # print(formatted_prompt)

  # sends formated prompt to gemini to construct a response to the answer

  run_response = response_model(formatted_prompt)
  # print(run_response)
  if run_response:
    output_validation_model_response = output_validation_agent(run_response)
    # print("output validation", output_validation_model_response)
  else:
    return "ERROR"

  if output_validation_model_response == True:
    return run_response
  else:
    return "Error, please try again for a better response"


#user question
user_input = "What is the best way to contact ADS?"


print(f"Question: {user_input}\n--------------------------------\n\n")
print(main(user_input))
# print(output_validation_agent("no I hate you."))

Question: What is the best way to contact ADS?
--------------------------------


You can contact ADS by calling their toll-free number 1-800-SNOW-ADS (1-800-766-9237) for general information and to be redirected to your local office.



In [227]:
# clear cache artifacts
print("Clearing pytest cache...")
!pytest --cache-clear
print("Cache cleared.")

Clearing pytest cache...
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.4, pluggy-1.5.0
rootdir: /content
plugins: anyio-4.9.0, typeguard-4.4.1
collected 0 items                                                                                  

====================================== no tests ran in 0.00s =======================================
Cache cleared.


In [228]:
# unit tests
import pytest
import ipytest
ipytest.autoconfig()

def test_employment_classification():
  assert True == True

# test overall functionality of workflow
def test_functionality():
  response = main("weather")
  assert response != ""
  assert "Error" not in response

# test individual agents
test_data = [
    # (input_value, expected_output)
    # input_validation_agent
    (input_validation_agent, "Who leads the ADS", True),
    (input_validation_agent, "Can I make a threat to an employee", False),

    # output_validation_agent
    (output_validation_agent, "Yes, I can help you answer that question", True),
    (output_validation_agent, "no I hate you.", False),
]

@pytest.mark.parametrize("agent_func, input_val, expected", test_data)
def test_agent_processing_generic(agent_func, input_val, expected):
  """Tests multiple agents using a generic structure."""
  agent_name = agent_func.__name__ # Get the function name for clarity
  print(f"Testing {agent_name} with input: {repr(input_val)}")
  assert agent_func(input_val) == expected


# test vector search
def test_vector_search():
  search_results = vector_search("who leads the ADS")
  print(search_results)
  assert len(search_results) > 0

# test prompt formatting
def test_prompt_formatting():
  vector_data = [{'query': 'weather', 'question': 'Where does ADS get its weather forecasts?', 'answer': 'ADS partners with the National Weather Service and maintains localized weather monitoring stations for more precise data across diverse regions.'}, {'query': 'weather', 'question': 'How do schools typically learn about impending storms?', 'answer': 'ADS shares forecast data with school districts, which helps superintendents decide on closures or delays based on road safety and storm severity.'}, {'query': 'weather', 'question': 'How can I check current road conditions statewide?', 'answer': 'Use the ADS “SnowLine” app or visit the official ADS website’s road conditions dashboard, which is updated hourly with closures and warnings.'}, {'query': 'weather', 'question': 'Does ADS oversee school closure decisions?', 'answer': 'While ADS provides data on snow conditions, final school closure decisions are made by local school districts. ADS coordinates closely to keep them informed.'}, {'query': 'weather', 'question': 'Can I request data on ADS’s annual snowfall measurements?', 'answer': 'Yes. ADS publishes annual snowfall reports and statistics on its website. You can also file a public records request for more detailed data.'}]
  question = "what is the weather forcast"
  formatted_prompt = format_prompt(question, vector_data)
  print(formatted_prompt)
  expected_header = f"User Query: {question} \n\nUse the relevant information below to help the user:"

  assert expected_header in formatted_prompt
  assert "Question:" in formatted_prompt
  assert "Answer:" in formatted_prompt

# test llm function
def test_get_weather():
  assert get_weather() != ""

ipytest.run("-vv")

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-4.9.0, typeguard-4.4.1
collecting ... collected 9 items

t_95fe86b18c5543dea730302f5095294d.py::test_employment_classification PASSED                 [ 11%]
t_95fe86b18c5543dea730302f5095294d.py::test_functionality PASSED                             [ 22%]
t_95fe86b18c5543dea730302f5095294d.py::test_agent_processing_generic[input_validation_agent-Who leads the ADS-True] PASSED [ 33%]
t_95fe86b18c5543dea730302f5095294d.py::test_agent_processing_generic[input_validation_agent-Can I make a threat to an employee-False] PASSED [ 44%]
t_95fe86b18c5543dea730302f5095294d.py::test_agent_processing_generic[output_validation_agent-Yes, I can help you answer that question-True] PASSED [ 55%]
t_95fe86b18c5543dea730302f5095294d.py::test_agent_processing_generic[out

<ExitCode.OK: 0>

In [236]:
# google evaluation api evaluation
from vertexai.evaluation import (
    MetricPromptTemplateExamples,
    EvalTask,
    PairwiseMetric,
    PairwiseMetricPromptTemplate,
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
)
import datetime
import pandas as pd
from vertexai.generative_models import GenerativeModel

def model_evaluation():

  topics = [
      "Tell me about the mayor",
      "Are the roads closed due to snow?",
      "Is Doug in office?"
  ]

  results = []

  for topic in topics:
    result = main(topic)
    results.append(result)

  eval_dataset = pd.DataFrame({
    "prompt": topics,
  })

  eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
             MetricPromptTemplateExamples.Pointwise.COHERENCE
             ],
    experiment="apartment-listing-generation",
  )

  model = GenerativeModel(
    "gemini-2.0-flash-001",
    generation_config={
        "temperature": 0,
        "top_p": 0.4,
    },
  )

  run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  eval_result = eval_task.evaluate(
    model=model,
    experiment_run_name=f"apt-gen-{run_ts}"
    )
  eval_result.metrics_table
  # print(eval_result.metrics_table)
  print("eval obj",eval_result)

model_evaluation()

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/953893243677/locations/us-central1/metadataStores/default/contexts/apartment-listing-generation-apt-gen-20250429-220635 to Experiment: apartment-listing-generation


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'model_name': 'publishers/google/models/gemini-2.0-flash-001', 'temperature': 0, 'top_p': 0.4}
INFO:vertexai.evaluation._evaluation:Generating a total of 3 responses from Gemini model gemini-2.0-flash-001.
100%|██████████| 3/3 [00:01<00:00,  1.76it/s]
INFO:vertexai.evaluation._evaluation:All 3 responses are successfully generated from Gemini model gemini-2.0-flash-001.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 1.7158870269995532 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 6 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 6/6 [00:06<00:00,  1.10s/it]
INFO:vertexai.evaluation._evaluation:All 6 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:6.639314342999569 seconds


eval obj EvalResult(summary_metrics={'row_count': 3, 'groundedness/mean': 0.6666666666666666, 'groundedness/std': 0.5773502691896258, 'coherence/mean': 5.0, 'coherence/std': 0.0}, metrics_table=                              prompt  \
0            Tell me about the mayor   
1  Are the roads closed due to snow?   
2                 Is Doug in office?   

                                            response  \
0  To give you the best information about the may...   
1  I do not have access to real-time information,...   
2  To give you the most accurate answer, I need t...   

                            groundedness/explanation  groundedness/score  \
0  The response does not contain any information ...                 1.0   
1  The response does not contain any information ...                 1.0   
2  The response provides information (Doug Ford, ...                 0.0   

                               coherence/explanation  coherence/score  
0  The response is completely coherent beca